# Decision Tree Regressor, predicts the gravity, 0 to 100 (float)
## O melhor classificador encontrado com scoring = neg_mean_squared_error:
DecisionTreeRegressor(criterion='poisson', max_depth=16, min_samples_leaf=8,
                      random_state=42)

### Resultados:
- MAE com dados de 800 vítimas: 2.102206<br />
Em média, as previsões estão erradas por cerca de 2.10 unidades da verdade.<br />
- MSE com dados de 800 vítimas: 11.301262<br />
Há algum nível de erro, com erros maiores afetando mais o valor total.<br />
- RMSE com dados de 800 vítimas: 3.361735<br />
O erro típico de previsão é de cerca de 3.36 unidades.<br />
- R² com dados de 800 vítimas: 0.956811<br />
O modelo explica aproximadamente 95.68% da variância dos dados.<br />

### Análises:
#### Overfitting:
O modelo pode estar sobreajustado aos dados de treinamento, resultando em um desempenho ligeiramente pior em dados não vistos.
#### Underfitting:
O modelo está apresentando um bom desempenho em todos os conjuntos de dados, embora o desempenho em dados não vistos seja ligeiramente pior.


In [34]:
import pandas as pd
import numpy as np
import joblib

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [35]:
# Lê o dataset de 4000 vítimas, X(qPa, pulso e frequencia respiratória), y(gravidade)

file_path = 'datasets/4000.txt'
X = []
y = []
# Ler e processar cada linhawithopen(file_path, 'r') as file:
with open(file_path, 'r') as file:
     for line in file:
         # Separar os valores por vírgula
         values = line.strip().split(',')
        
         # Extrair as características (colunas 1 a 6) e o rótulo (última coluna)
         features = list(map(float, values[3:6]))  # Converta as características para float
         #label = int(values[7])  # Gravity class - Converta o rótulo para inteiro# Adicionar as características e rótulo às listas
         #label = label - 1  # Gravity class - Ajustar o rótulo para começar em 0
         label = float(values[6])
         X.append(features)
         y.append(label)


In [36]:
# Lê o dataset de 4000 vítimas, X2(qPa, pulso e frequencia respiratória), y2(gravidade)

file_path2 = 'datasets/800.txt'
X2 = []
y2 = []
# Ler e processar cada linhawithopen(file_path, 'r') as file:
with open(file_path2, 'r') as file:
     for line in file:
         # Separar os valores por vírgula
         values = line.strip().split(',')
        
         # Extrair as características (colunas 1 a 6) e o rótulo (última coluna)
         features = list(map(float, values[3:6]))  # Converta as características para float
         #label = int(values[7])  # Gravity class - Converta o rótulo para inteiro# Adicionar as características e rótulo às listas
         #label = label - 1  # Gravity class - Ajustar o rótulo para começar em 0
         label = float(values[6])
         X2.append(features)
         y2.append(label)

In [37]:
# Cria o dataframe de 4000 vitimas SEM a gravidade
df = pd.DataFrame(X, columns = ['qPa','pulse','respiratory_freq'])
print("Amostras iniciais do dataframe")
print(df.head(3)) # imprime 3 amostras do dataframe

Amostras iniciais do dataframe
        qPa       pulse  respiratory_freq
0  8.652291  122.324349          5.623564
1  8.733333  135.824333         12.787053
2  8.733333   97.100546          1.553286


In [38]:
# Cria o dataframe de 800 vitimas SEM a gravidade
df2 = pd.DataFrame(X2, columns = ['qPa','pulse','respiratory_freq'])
print("Amostras iniciais do dataframe 800")
print(df2.head(3)) # imprime 3 amostras do dataframe

Amostras iniciais do dataframe 800
        qPa       pulse  respiratory_freq
0  8.721732  160.904175         20.926336
1  8.733333  134.454047         17.972046
2  8.614814   86.302237          4.168373


In [39]:
## Dividir o dataset em treinamento/validação e teste
## É usual designar a entrada por X e a saída por y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, random_state=42)#utilizado random state 42 para manter constante
print(f"Tamanho total do dataset: {len(X)}\n")
print(f"Dados de treinamento X ({len(X_train)}):\n{X_train[:3]} ...")
print(f"Dados de treinamento y:({len(y_train)})\n {y_train[:3]} ...")
print("---")
print(f"Dados de teste   X ({len(X_test)}):\n{X_test[:3]} ...")
print(f"Dados de teste   y:({len(y_test)})\n {y_test[:3]} ...")

Tamanho total do dataset: 4000

Dados de treinamento X (3000):
[[4.358461, 145.99551, 15.178772], [-4.385294, 155.005138, 15.287787], [-8.709306, 16.292285, 17.436375]] ...
Dados de treinamento y:(3000)
 [50.034547, 67.993565, 35.241338] ...
---
Dados de teste   X (1000):
[[4.685739, 24.375112, 18.209474], [-4.399409, 82.247291, 7.246955], [8.733333, 113.558415, 17.34836]] ...
Dados de teste   y:(1000)
 [39.082517, 49.826235, 49.382022] ...


In [40]:
# Parameters' definition
parameters = {
    'criterion': ['poisson'], #, 'absolute_error', 'squared_error', 'friedman_mse'],
    'max_depth': [2, 4, 8, 16],
    #'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [8, 16, 32, 64]
}
#scoring_options = {
#    'MSE': 'neg_mean_squared_error',
#    'MAE': 'neg_mean_absolute_error',
#    'RMSE': 'neg_root_mean_squared_error',
#    'R2': 'r2'
#}

# instantiate model
# random_state = 42 to be deterministic
model = DecisionTreeRegressor(random_state=42)

# Best model found: DecisionTreeRegressor(criterion='poisson', max_depth=16, min_samples_leaf=8,
#                      random_state=42)

# grid search using cross-validation
# cv = 3 is the number of folds
# scoring = 'accuracy' the metric for chosing the best model
clf = GridSearchCV(model, parameters, cv=3, scoring='neg_mean_squared_error', verbose=4)
clf.fit(X_train, y_train)

# the best tree according to the accuracy score
best = clf.best_estimator_
print("\n* Melhor classificador *")
print(clf.best_estimator_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV 1/3] END criterion=poisson, max_depth=2, min_samples_leaf=8;, score=-160.237 total time=   0.0s
[CV 2/3] END criterion=poisson, max_depth=2, min_samples_leaf=8;, score=-159.910 total time=   0.0s
[CV 3/3] END criterion=poisson, max_depth=2, min_samples_leaf=8;, score=-168.057 total time=   0.0s
[CV 1/3] END criterion=poisson, max_depth=2, min_samples_leaf=16;, score=-160.237 total time=   0.0s
[CV 2/3] END criterion=poisson, max_depth=2, min_samples_leaf=16;, score=-159.910 total time=   0.0s
[CV 3/3] END criterion=poisson, max_depth=2, min_samples_leaf=16;, score=-168.057 total time=   0.0s
[CV 1/3] END criterion=poisson, max_depth=2, min_samples_leaf=32;, score=-160.237 total time=   0.0s
[CV 2/3] END criterion=poisson, max_depth=2, min_samples_leaf=32;, score=-159.910 total time=   0.0s
[CV 3/3] END criterion=poisson, max_depth=2, min_samples_leaf=32;, score=-168.057 total time=   0.0s
[CV 1/3] END criterion=poisson, m

In [41]:
# Predicoes
# com dados do treinamento
y_pred_train = best.predict(X_train)
mae_train = mean_absolute_error(y_train, y_pred_train)
mse_train = mean_squared_error(y_train, y_pred_train)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, y_pred_train)

print(f"MAE com dados de treino: {mae_train:.6f}")
print(f"MSE com dados de treino: {mse_train:.6f}")
print(f"RMSE com dados de treino: {rmse_train:.6f}")
print(f"R² com dados de treino: {r2_train:.6f}")

# Predições com dados de teste
y_pred_test = best.predict(X_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y_test, y_pred_test)

print(f"MAE com dados de teste: {mae_test:.6f}")
print(f"MSE com dados de teste: {mse_test:.6f}")
print(f"RMSE com dados de teste: {rmse_test:.6f}")
print(f"R² com dados de teste: {r2_test:.6f}")

# Predições com dados do dataset de 800 vítimas
y_pred_test2 = best.predict(X2)
mae_test2 = mean_absolute_error(y2, y_pred_test2)
mse_test2 = mean_squared_error(y2, y_pred_test2)
rmse_test2 = np.sqrt(mse_test2)
r2_test2 = r2_score(y2, y_pred_test2)

print(f"MAE com dados de 800 vítimas: {mae_test2:.6f}")
print(f"MSE com dados de 800 vítimas: {mse_test2:.6f}")
print(f"RMSE com dados de 800 vítimas: {rmse_test2:.6f}")
print(f"R² com dados de 800 vítimas: {r2_test2:.6f}")

MAE com dados de treino: 1.515655
MSE com dados de treino: 5.660392
RMSE com dados de treino: 2.379158
R² com dados de treino: 0.977777
MAE com dados de teste: 1.984884
MSE com dados de teste: 9.964327
RMSE com dados de teste: 3.156632
R² com dados de teste: 0.961956
MAE com dados de 800 vítimas: 2.102206
MSE com dados de 800 vítimas: 11.301262
RMSE com dados de 800 vítimas: 3.361735
R² com dados de 800 vítimas: 0.956811


In [42]:
import os

#Salvando o modelo
if not os.path.exists('dt_regressor_models'):
    os.mkdir('dt_regressor_models')

joblib.dump(best, 'dt_regressor_models/best_regressor.pkl')
with open("dt_regressor_models/models.txt", "w") as file:
    file.write(f'Config: {str(best)}\nMAE com dados de 800 vítimas: {mae_test2:.6f}\nMSE com dados de 800 vítimas: {mse_test2:.6f}\nRMSE com dados de 800 vítimas:{rmse_test2:.6f}\nR² com dados de 800 vítimas: {r2_test2:.6f}')

In [43]:
#Para carregar o modelo

# Carregar o modelo salvo
model = joblib.load('dt_regressor_models/best_regressor.pkl')

# Agora você pode usar o modelo carregado para fazer previsões
y_pred_test2 = model.predict(X2)
mae_test2 = mean_absolute_error(y2, y_pred_test2)
mse_test2 = mean_squared_error(y2, y_pred_test2)
rmse_test2 = np.sqrt(mse_test2)
r2_test2 = r2_score(y2, y_pred_test2)

print(f"MAE com dados de 800 vítimas: {mae_test2:.6f}")
print(f"MSE com dados de 800 vítimas: {mse_test2:.6f}")
print(f"RMSE com dados de 800 vítimas: {rmse_test2:.6f}")
print(f"R² com dados de 800 vítimas: {r2_test2:.6f}")

MAE com dados de 800 vítimas: 2.102206
MSE com dados de 800 vítimas: 11.301262
RMSE com dados de 800 vítimas: 3.361735
R² com dados de 800 vítimas: 0.956811
